In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import nilearn
from nilearn import datasets, surface
from scipy.stats import zscore, spearmanr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import MDS
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from statsmodels.stats.multitest import fdrcorrection
from brainspace.datasets import load_parcellation, load_conte69
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels 
from brainspace.null_models import SpinPermutations
from enigmatoolbox import datasets
from enigmatoolbox.utils.parcellation import parcel_to_surface
from enigmatoolbox.plotting import plot_cortical
import scikit_posthocs as sp
from palettable.cartocolors.qualitative import Vivid_4
surf_lh, surf_rh = load_conte69()
outpath='path/out/'

In [ ]:
#KIDB preprocessing
ki_database=pd.read_csv('KiDatabase.csv', encoding='mac-roman', usecols=['Name','Unigene',' Ligand Name','species','source', 'ki Note','ki Val', 'Reference'])
ki_database_human=ki_database[ki_database['species'] == 'HUMAN']
def data_driven_selector(inp):
    ki_database_human=inp
    #standardize input and filter for >
    ki_database_human[' Ligand Name']=ki_database_human[' Ligand Name'].str.capitalize()
    ki_database_human=ki_database_human[ki_database_human['ki Note'] != '>']
    ki_database_human=ki_database_human[ki_database_human['ki Val'] < 10000]
    substances=['Agomelatine', 'Amitriptyline', 'Bupropion', 'Citalopram',
       'Clomipramine', 'Desvenlafaxine', 'Duloxetine', 'Escitalopram',
       'Fluoxetine', 'Fluvoxamine', 'Milnacipran', 'Mirtazapine',
       'Nefazodone', 'Paroxetine', 'Reboxetine', 'Sertraline',
       'Trazodone', 'Venlafaxine']
    d={substance:{} for substance in substances}
    for substance in substances:
        subset=ki_database_human[ki_database_human[' Ligand Name'] == substance]
        molecules=np.unique(subset['Name'])
        for molecule in molecules:
            d[substance][molecule]=np.mean(subset[subset['Name'] == molecule]['ki Val'])
    return d

d=data_driven_selector(ki_database_human)
dd_ad=pd.DataFrame(d)

#manual unification
synonyms_dict={'5-HTT':['5-HT Transporter','SERT'], '5-HT1A':['5-HT1A'],'5-HT1B':['5-HT1B'],
              '5-HT2A':['5-HT2A', '5-HT2'], '5-HT2B':['5-HT2B', '5-HT2'], '5-HT2C':['5-HT2C', '5-HT2'],
               '5-HT6':['5-HT6'],'NET':['Norepinephrine transporter'],
               'alpha1':['Alpha 1 Adrenergic Receptor', 'adrenergic Alpha1', 'adrenergic Alpha1A'],
              'alpha2':['Alpha 2 Adrenergic Receptor', 'adrenergic Alpha2', 'adrenergic Alpha2A','adrenergic Alpha2B','adrenergic Alpha2C'],
              'DAT':['Dopamine Transporter'], 'D1':['DOPAMINE D1','D1'],'D2':['DOPAMINE D2','D2'],
              'D3':['DOPAMINE D3','D3'], 'D4':['DOPAMINE D4','D4'], 'D5':['DOPAMINE D5'],
              'H1':['HISTAMINE H1', 'H1'],'H2':['HISTAMINE H2', 'H2'], 'H3':['HISTAMINE H3', 'H3'],
              'H4':['HISTAMINE H4', 'H4'],
               'M1':['Muscarinic Acetylcholine Receptor','Cholinergic, muscarinic','Cholinergic, muscarinic M1'],
              'M2':['Muscarinic Acetylcholine Receptor','Cholinergic, muscarinic','Cholinergic, muscarinic M2'],
              'M3':['Muscarinic Acetylcholine Receptor','Cholinergic, muscarinic','Cholinergic, muscarinic M3'],
              'M4':['Muscarinic Acetylcholine Receptor','Cholinergic, muscarinic','Cholinergic, muscarinic M4'],
              'M5':['Muscarinic Acetylcholine Receptor','Cholinergic, muscarinic','Cholinergic, muscarinic M5']}
unified={unificator:{} for unificator in synonyms_dict.keys()}
for key, val in synonyms_dict.items():
    extract=dd_ad.T[val].T.mean()
    unified[key]=dict(extract)

dd_unified=pd.DataFrame(unified)
dd_unified.fillna(value=10000, inplace=True) #fill missing data and cap Ki values at 10000
dd_unified.to_csv('kidb_ad.csv')

In [ ]:
#load data
#antidepressant data
dd_unified=pd.read_csv('kidb_ad.csv', index_col=0) #Inhibition coefficients from human sources averaged across KiDB entries per target
cipriani=pd.read_csv('efficacy_data.csv', index_col=0) #Efficacy data from https://doi.org/10.1016/s0140-6736(17)32802-7

#neurochemistry
receptors_schaefer=pd.read_csv('100Parcels7Networks_receptorprofiles.csv', index_col=0) #Chemoarchitectural data as described in https://doi.org/10.7554/eLife.83843
receptors_schaefer=receptors_schaefer.apply(zscore)

#enigma
d={}
sum_stats=datasets.load_summary_stats('depression')
CT=sum_stats['CortThick_case_vs_controls_adult']
CT=CT['d_icv']
d['depression_all']=CT
enigma_df=pd.DataFrame(d)

#annotations
networks=np.load('100parcels_7networks.npy') #7 Yeo networks in 100 Schaefer parcels as per https://doi.org/10.1093/cercor/bhx179
types_100=np.load('mesulam_100parc.npy') #Division of the cortex into four cortical types by Mesulam as per https://doi.org/10.1093/brain/121.6.1013

In [ ]:
#Figure 1A
sns.set(font_scale=2.55)
fig=sns.clustermap(np.log(dd_unified), method='ward', figsize=(18,14), tree_kws=dict(linewidths=1.6))
fig.savefig(outpath + 'FIG_1A.jpeg')
plt.close()

#Cluster validation supplement - supplementary figures 3 and 4
for i in ['single','complete','average','weighted','centroid','median','ward']:
    for j,k in enumerate([np.log(dd_unified), dd_unified]):
        fig = sns.clustermap(k, method=i, figsize=(18,14), tree_kws=dict(linewidths=1.6))
        plt.tight_layout()
        if j == 0:
            fig.savefig(outpath + 'ClusVal_log_{}.jpeg'.format(i))
            plt.close()
        else:
            fig.savefig(outpath + 'ClusVal_nolog_{}.jpeg'.format(i))
            plt.close()

In [ ]:
#Figure 1B
X=np.log(dd_unified).values
kmeans = KMeans(n_clusters=3, random_state=0, init='k-means++', n_init=10).fit(X)
color_dict={0 : 'cyan', 1 : 'y', 2 : 'violet'}
colors=[color_dict[i] for i in kmeans.labels_]
mds=MDS(random_state=1)
mds_fit=mds.fit_transform(np.log(dd_unified))
sns.set_style('white')
fig, ax=plt.subplots(figsize=(18,14))
plt.scatter(mds_fit[:,0], mds_fit[:,1], s=320, c=colors)
labels_list=['AGO', 'AMI', 'BUP', 'CIT', 'CLO', 'DVEN', 'DUL', 'ECIT', 'FLUO','FLUV','MIL','MIR','NEF','PAR','REB','SER',
            'TRA','VEN']
ax.set_ylabel('Dimension 2', fontsize=32)
ax.set_xlabel('Dimension 1', fontsize=32)
ax.tick_params(labelsize=28)
for i, txt in enumerate(labels_list):
    ax.annotate(txt, (mds_fit[i,0], mds_fit[i,1]), fontsize=32)
plt.tight_layout()
fig.savefig(outpath + 'FIG_1B.jpeg')
plt.close()

In [ ]:
# Figure 1C
combined=dd_unified.merge(cipriani, left_index=True, right_on='compound')
corr, pval=spearmanr(dd_unified, combined['efficacy'])
print('fdr-corrected statistical significance at alpha = 0.05 using the benjamini-hochberg method:')
print(list(zip(dd_unified.index, fdrcorrection(pval[-1])[0])))
sns.set_style('ticks')
fig, ax = plt.subplots(figsize=(30,8))
interest=corr[-1][:-1]
color=['lightskyblue' if x > 0.05 else 'dodgerblue' for x in pval[-1][:-1]]
ax.bar(range(len(interest)), interest, color=color, width=0.9, linewidth=1)
ax.set_xticks(range(len(interest)))
ax.set_xticklabels(dd_unified.columns, rotation=45, ha='center')
ax.set_ylabel("spearman's r", fontsize=36)
ax.tick_params(labelsize=36)
plt.tight_layout()
fig.savefig(outpath + 'FIG_1C.jpeg')
plt.close()

In [ ]:
# Figure 2A and Supplementary Figure 4B
labeling = load_parcellation('schaefer', scale=100, join=True)
labels_list=['AGO', 'AMI', 'BUP', 'CIT', 'CLO', 'DVEN', 'DUL', 'ECIT', 'FLUO','FLUV','MIL','MIR','NEF','PAR','REB','SER',
            'TRA','VEN']
affi_max=dd_unified[['5-HTT','NET','DAT','5-HT1A','5-HT1B','5-HT2A','5-HT6','D1','D2', 'M1','H3']]
max_overlap=receptors_schaefer[['5HTT','NAT','DAT','5HT1a','5HT1b','5HT2a','5HT6','D1','D2','M1','H3']]
fact=1 / affi_max
fact=fact.values
red_rec_array=max_overlap.values
weighted_total=red_rec_array @ fact.T
weighted_total_df=pd.DataFrame(weighted_total, index=max_overlap.index, columns=affi_max.index)
weighted_total_df.index=networks
for i in range(len(weighted_total[0])):
    interest=weighted_total[:,i]
    label=labels_list[i]
    grad1_100=map_to_labels(interest, labeling, mask=labeling != 0, fill=np.nan)
    plot_hemispheres(surf_lh, surf_rh, array_name=grad1_100, size= (1000,600),color_bar=False, label_text=[label],zoom=1.4,
                      layout_style='grid',
                 cmap='viridis', screenshot=True, transparent_bg = True, filename=outpath + 'FIG_2A_{}.png'.format(label))

In [ ]:
# Figure 2B
weighted_total_z=weighted_total_df.apply(zscore)
yeo_colours = np.array([[120 ,18 ,134],[ 70, 130, 180],[ 0, 118, 14],[196, 58, 250],[ 220, 248, 164],[230, 148, 34,], [205, 62, 78]]) / 256
for i in ['Amitriptyline','Mirtazapine','Duloxetine']:
    fig, ax=plt.subplots(figsize=(8,6))
    sns.boxplot(data=weighted_total_z, x=weighted_total_df.index, y=i, palette=yeo_colours)
    ax.set_xticklabels(['vis','som-\nmot','dors\natt','vent\natt','limb','cont','def'])
    ax.tick_params(labelsize=26)
    ax.set_ylabel(i, fontsize=26)
    ax.set_xlabel('')
    plt.tight_layout()
    fig.savefig(outpath + 'FIG_2B_FC_{}.jpeg'.format(i))
    plt.close()

weighted_total_z['Mesulam']=types_100
col_types={'paralimbic':Vivid_4.mpl_colors[3],'unimodal':Vivid_4.mpl_colors[1],
          'idiotypic':Vivid_4.mpl_colors[0],'heteromodal':Vivid_4.mpl_colors[2]}
for i in ['Amitriptyline','Mirtazapine','Duloxetine']:
    fig, ax=plt.subplots(figsize=(8,6))
    sns.boxplot(y=i, x='Mesulam', data=weighted_total_z, order=['idiotypic','unimodal',
                                                               'heteromodal','paralimbic'], palette=col_types ,width=0.65)
    ax.set_xticks([0,1,2,3])
    ax.tick_params(labelsize=26)
    ax.set_xticklabels(['idio-\ntypic','uni-\nmodal','hetero-\nmodal','para-\nlimbic'])
    ax.tick_params(labelsize=26)
    ax.set_ylabel(i, fontsize=26)
    ax.set_xlabel('')
    plt.tight_layout()
    fig.savefig(outpath + 'FIG_2B_CT_{}.jpeg'.format(i))
    plt.close()

In [ ]:
# Figure 2C
fsavg=nilearn.datasets.fetch_surf_fsaverage('fsaverage5')
sphere_lh_fs=nilearn.surface.load_surf_mesh(fsavg['sphere_left'])[0]
sphere_rh_fs=nilearn.surface.load_surf_mesh(fsavg['sphere_right'])[0]
sp_fs = SpinPermutations(n_rep=1000, random_state=0)
sp_fs.fit(sphere_lh_fs, points_rh=sphere_rh_fs)

def spin(inp, grad, spin_grad):
    r_spin = np.empty(1000)
    mask = ~np.isnan(grad) & ~ np.isnan(inp)
    r_obs = spearmanr(grad[mask], inp[mask])[0]
    #print(r_obs)
    for i, perm in enumerate(spin_grad):
        mask_rot = mask & ~np.isnan(perm)  # Remove midline
        r_spin[i] = spearmanr(perm[mask_rot], inp[mask_rot])[0]
    pv_spin = np.mean(np.abs(r_spin) >= np.abs(r_obs))
    return [r_obs, pv_spin]

def permute(inp):
    half=int(len(inp) / 2)
    left=inp[half:]
    right=inp[:half]
    return np.hstack(sp_fs.randomize(left, right))

enigma_surf=parcel_to_surface(enigma_df['depression_all'], 'aparc_fsa5')
permuted=permute(enigma_surf)
result_dk_schaefer={}
for i in weighted_total_df.columns:
    substance=weighted_total_df[i].values
    substance_surf=parcel_to_surface(substance, 'schaefer_100_fsa5')
    result_dk_schaefer[i]=spin(substance_surf, enigma_surf, permuted)
pvals=np.array(list(result_dk_schaefer.values()))[:,1]
print('fdr-corrected statistical significance at alpha = 0.05 using the benjamini-hochberg method:')
print(list(zip(dd_unified.index, fdrcorrection(pvals)[0])))


sns.set_style('ticks')
fig, ax = plt.subplots(figsize=(6,14))

plotting=pd.DataFrame(result_dk_schaefer).T
spear=list(plotting[0])[::-1]
pval=list(plotting[1])[::-1]
color=['lightskyblue' if x > 0.05 else 'dodgerblue' for x in pval]

ax.barh(range(len(plotting)), width=spear, color=color)
ax.set_yticks(range(len(plotting)))
ax.set_yticklabels(labels_list[::-1])
ax.set_xlabel("spearman's r", fontsize=32)
ax.tick_params(labelsize=32)
ax.set_xlim(-0.5, 0.25)
plt.tight_layout()
fig.savefig(outpath + 'FIG_2C.jpeg')
plt.close()

In [ ]:
#Supplementary Figure 1
"""
Adapted from:
https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
"""
X=np.log(dd_unified).values

range_n_clusters = [2, 3, 4, 5]

for n_clusters in range_n_clusters:
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set_xlim([0, 1])
    ax.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i), fontsize=22)

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax.set_xlabel("Silhouette coefficient values", fontsize=24)

    # The vertical line for average silhouette score of all the values
    lin=ax.axvline(x=silhouette_avg, color="red", linestyle="--")
    lin.set_label(f'{silhouette_avg:.3f}')
    ax.legend(fontsize=22)

    ax.set_yticks([])  # Clear the yaxis labels / ticks
    ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.tick_params(labelsize=22)

    plt.suptitle(" n = %d clusters" % n_clusters,fontsize=28)
    plt.tight_layout()
    fig.savefig(outpath + 'SFIG_1_{}.jpeg'.format(n_clusters))
    plt.close()

    

In [ ]:
#Supplemental Figure 2
eff=pd.merge(dd_unified, cipriani, left_index=True, right_on='compound')
eff['cluster']=kmeans.labels_
eff=eff[['compound','cluster','efficacy']]
fig, ax=plt.subplots(figsize=(10,10))
color_dict={0 : 'yellow', 1 : 'violet', 2 :'cyan'}
colors=[color_dict[i] for i in kmeans.labels_]
sns.swarmplot(data=eff, x='cluster', y='efficacy', color='black')
sns.boxplot(data=eff, x='cluster', y='efficacy', palette=['cyan', 'y', 'violet'])
ax.set_ylabel('efficacy', fontsize=24)
ax.set_xlabel('cluster', fontsize=24)
ax.tick_params(labelsize=22)
fig.savefig(outpath + 'SFIG_2A.jpeg')
plt.close()

#correlation
combined=dd_unified.merge(eff, left_index=True, right_on='compound')
index=combined['cluster'] == 1
index=index.values
combined=combined[index]
corr, pval=spearmanr(dd_unified[index], combined['efficacy'])
sns.set_style('ticks')
fig, ax = plt.subplots(figsize=(6, 10))
interest=corr[-1][:-1]
color=['lightskyblue' if x > 0.05 else 'dodgerblue' for x in pval[-1][:-1]]
ax.barh(range(len(interest)), width=interest, color=color, linewidth=1)
ax.set_yticks(range(len(interest)))
ax.set_yticklabels(dd_unified.columns)
ax.set_xlabel("spearman's r", fontsize=26)
ax.tick_params(labelsize=26)
plt.tight_layout()
fig.savefig(outpath + 'SFIG_2B.jpeg')
plt.close()

In [ ]:
#Supplementary Figure 5
fig, ax = plt.subplots(figsize=(14,8))
sns.set_theme(font_scale=2.5, style='ticks')
corr=weighted_total_df[['Amitriptyline', 'Clomipramine','Mirtazapine','Nefazodone','Trazodone','Agomelatine','Bupropion',
                   'Reboxetine','Milnacipran','Desvenlafaxine','Fluvoxamine', 'Paroxetine','Sertraline','Duloxetine',
                   'Citalopram','Escitalopram', 'Fluoxetine','Venlafaxine']].corr('spearman')
mask = np.triu(np.ones_like(corr, dtype=bool))
adj_labels=['AMI','CLO','MIR','NEF','TRA','AGO','BUP','REB','MIL','DVEN','FLUV','PAR','SER','DUL','CIT','ECIT','FLUO','VEN']
sns.heatmap(corr, mask=mask, cmap='RdBu', vmin=-1, vmax=1, xticklabels=adj_labels, yticklabels=adj_labels)
plt.tight_layout()
fig.savefig(outpath + 'SFIG_5A.jpeg')
plt.close()